<a href="https://colab.research.google.com/github/RifatMuhtasim/Data_Science/blob/main/Model_Training/Optuna_Regression_Hyperparameter_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.datasets import load_diabetes

# Load the Diabetes dataset
diabetes = load_diabetes()

df = pd.DataFrame(data=diabetes.data, columns=diabetes.feature_names)
df['target'] = diabetes.target
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [3]:
X = df.drop(['target'], axis="columns")
y = df['target']

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

def objective(X, y):
    reg = LinearRegression()
    mse_scores = -cross_val_score(
        reg, X, y, scoring='neg_mean_squared_error', cv=5, n_jobs=-1
    )  # Get negative mean squared error scores.
    rmse_scores = np.sqrt(mse_scores)
    return rmse_scores.mean()

rmse_result = objective(X, y)
print(f"RMSE: {rmse_result}")

RMSE: 54.69179097275793


In [5]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

def objective(X, y):
    reg = XGBRegressor()
    mse_scores = -cross_val_score(
        reg, X, y, scoring='neg_mean_squared_error', cv=5, n_jobs=-1
    )  # Get negative mean squared error scores.
    rmse_scores = np.sqrt(mse_scores)
    return rmse_scores.mean()

rmse_result = objective(X, y)
print(f"RMSE: {rmse_result}")

RMSE: 63.02569539868239


In [6]:
# Check Multiple Model
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor


def objective(X, y, model):
    reg = model

    mse_scores = -cross_val_score(
        reg, X, y, scoring='neg_mean_squared_error', cv=5, n_jobs=-1
    )  # Get negative mean squared error scores.
    rmse_scores = np.sqrt(mse_scores)
    return rmse_scores.mean()


models = {'Linear_Regression':  LinearRegression(),
                    'Random_Forest': RandomForestRegressor(),
                    'Decision_Tree': DecisionTreeRegressor(),
                    'XGB_Regressor': XGBRegressor(),
                    'SVM': SVR(),
                    'KNearest_Neighbors': KNeighborsRegressor()}

for model_name, model in models.items():
    result = objective(X, y, model=model)
    print(f"{model_name} RMSE is: ", result)

Linear_Regression RMSE is:  54.69179097275793
Random_Forest RMSE is:  58.33538766551059
Decision_Tree RMSE is:  82.89974859596767
XGB_Regressor RMSE is:  63.02569539868239
SVM RMSE is:  70.44355911952458
KNearest_Neighbors RMSE is:  60.126361947859564


# Optuna


In [7]:
try:
    import optuna
except ImportError:
    !pip install --quiet optuna
    import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 2.2 MB/s eta 0:00:00


In [8]:
import optuna
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np

def LinearRegression_Optuna(X, y):
    def objective(trial, X, y):
        params = {
            'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False])
        }

        clf = LinearRegression(**params)
        mse_scores = -cross_val_score(clf, X, y, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
        rmse_scores = np.sqrt(mse_scores)
        return rmse_scores.mean()

    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective(trial, X, y), n_trials=100)
    trial = study.best_trial
    return trial

result = LinearRegression_Optuna(X, y)
print(f"Linear Regression RMSE: {result.value}")
print(f"Best Hyperparameters: {result.params}")

[I 2024-05-07 21:35:33,723] A new study created in memory with name: no-name-1783b691-c7bf-4ca8-8d4f-b28c7684f861
[I 2024-05-07 21:35:33,810] Trial 0 finished with value: 54.69179097275793 and parameters: {'fit_intercept': True}. Best is trial 0 with value: 54.69179097275793.
[I 2024-05-07 21:35:33,895] Trial 1 finished with value: 54.69179097275793 and parameters: {'fit_intercept': True}. Best is trial 0 with value: 54.69179097275793.
[I 2024-05-07 21:35:33,976] Trial 2 finished with value: 54.69179097275793 and parameters: {'fit_intercept': True}. Best is trial 0 with value: 54.69179097275793.
[I 2024-05-07 21:35:34,052] Trial 3 finished with value: 165.91150716546554 and parameters: {'fit_intercept': False}. Best is trial 0 with value: 54.69179097275793.
[I 2024-05-07 21:35:34,133] Trial 4 finished with value: 54.69179097275793 and parameters: {'fit_intercept': True}. Best is trial 0 with value: 54.69179097275793.
[I 2024-05-07 21:35:34,211] Trial 5 finished with value: 54.691790972

Linear Regression RMSE: 54.69179097275793
Best Hyperparameters: {'fit_intercept': True}


In [9]:
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np

def XGBRegressor_Optuna(X, y):
    def objective(trial, X, y):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 100, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 100, log=True),
            'random_state': 42,
            'n_jobs': -1
        }

        clf = XGBRegressor(**params)
        mse_scores = -cross_val_score(clf, X, y, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
        rmse_scores = np.sqrt(mse_scores)
        return rmse_scores.mean()

    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective(trial, X, y), n_trials=100)
    trial = study.best_trial
    return trial

result = XGBRegressor_Optuna(X, y)
print(f"XGBoost Regressor RMSE: {result.value}")
print(f"Best Hyperparameters: {result.params}")

[I 2024-05-07 21:35:39,512] A new study created in memory with name: no-name-564c2bb5-a880-4ed2-914f-1e35de446237
[I 2024-05-07 21:35:43,948] Trial 0 finished with value: 59.30507412984574 and parameters: {'n_estimators': 612, 'max_depth': 7, 'learning_rate': 0.0640843958769893, 'subsample': 0.7675547122959264, 'colsample_bytree': 0.7547695691830059, 'reg_alpha': 0.005528656059517041, 'reg_lambda': 1.5915514471250152}. Best is trial 0 with value: 59.30507412984574.
[I 2024-05-07 21:35:47,169] Trial 1 finished with value: 68.76549206246318 and parameters: {'n_estimators': 235, 'max_depth': 7, 'learning_rate': 0.0014661370908162995, 'subsample': 0.817577518970684, 'colsample_bytree': 0.7812913652973004, 'reg_alpha': 59.29388969855162, 'reg_lambda': 1.2206775212227874e-05}. Best is trial 0 with value: 59.30507412984574.
[I 2024-05-07 21:36:03,740] Trial 2 finished with value: 59.99383578900506 and parameters: {'n_estimators': 861, 'max_depth': 10, 'learning_rate': 0.02076086748775769, 'su

XGBoost Regressor RMSE: 55.84687802146559
Best Hyperparameters: {'n_estimators': 474, 'max_depth': 3, 'learning_rate': 0.007109396406721442, 'subsample': 0.6005558538842404, 'colsample_bytree': 0.7943528179004098, 'reg_alpha': 0.0016871761348989979, 'reg_lambda': 0.008713978434717428}


In [10]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np

def DecisionTreeRegressor_Optuna(X, y):
    def objective(trial, X, y):
        params = {
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
            'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2', None]),
            'random_state': 42
        }

        clf = DecisionTreeRegressor(**params)
        mse_scores = -cross_val_score(clf, X, y, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
        rmse_scores = np.sqrt(mse_scores)
        return rmse_scores.mean()

    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective(trial, X, y), n_trials=100)
    trial = study.best_trial
    return trial

result = DecisionTreeRegressor_Optuna(X, y)
print(f"DecisionTreeRegressor RMSE: {result.value}")
print(f"Best Hyperparameters: {result.params}")

[I 2024-05-07 21:38:27,604] A new study created in memory with name: no-name-21d999d3-b8ce-4b03-b0e3-ef2ec419d375
[I 2024-05-07 21:38:27,648] Trial 0 finished with value: 65.24080279623422 and parameters: {'max_depth': 11, 'min_samples_split': 11, 'min_samples_leaf': 18, 'max_features': 'log2'}. Best is trial 0 with value: 65.24080279623422.
[I 2024-05-07 21:38:27,704] Trial 1 finished with value: 62.91332428686553 and parameters: {'max_depth': 4, 'min_samples_split': 18, 'min_samples_leaf': 6, 'max_features': 'auto'}. Best is trial 1 with value: 62.91332428686553.
[I 2024-05-07 21:38:27,744] Trial 2 finished with value: 68.68553729590494 and parameters: {'max_depth': 14, 'min_samples_split': 10, 'min_samples_leaf': 5, 'max_features': 'log2'}. Best is trial 1 with value: 62.91332428686553.
[I 2024-05-07 21:38:27,791] Trial 3 finished with value: 67.3601530951007 and parameters: {'max_depth': 18, 'min_samples_split': 17, 'min_samples_leaf': 8, 'max_features': 'log2'}. Best is trial 1 wi

DecisionTreeRegressor RMSE: 60.96358634871011
Best Hyperparameters: {'max_depth': 4, 'min_samples_split': 18, 'min_samples_leaf': 20, 'max_features': None}


In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import optuna
import numpy as np

def RandomForestRegressor_Optuna(X, y):
    def objective(trial, X, y):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
            'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2']),
            'random_state': 42
        }

        clf = RandomForestRegressor(**params)
        mse_scores = -cross_val_score(clf, X, y, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
        rmse_scores = np.sqrt(mse_scores)
        return rmse_scores.mean()

    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective(trial, X, y), n_trials=100)
    trial = study.best_trial
    return trial

result = RandomForestRegressor_Optuna(X, y)
print(f"RandomForestRegressor RMSE: {result.value}")
print(f"Best Hyperparameters: {result.params}")


[I 2024-05-07 21:38:35,939] A new study created in memory with name: no-name-0919b024-a09c-4223-b154-adccf399a0a4
[I 2024-05-07 21:38:37,768] Trial 0 finished with value: 56.43222255515438 and parameters: {'n_estimators': 200, 'max_depth': 11, 'min_samples_split': 16, 'min_samples_leaf': 7, 'max_features': 'auto'}. Best is trial 0 with value: 56.43222255515438.
[I 2024-05-07 21:38:43,854] Trial 1 finished with value: 57.234815502874675 and parameters: {'n_estimators': 800, 'max_depth': 4, 'min_samples_split': 16, 'min_samples_leaf': 20, 'max_features': 'sqrt'}. Best is trial 0 with value: 56.43222255515438.
[I 2024-05-07 21:38:48,673] Trial 2 finished with value: 56.10061975866422 and parameters: {'n_estimators': 700, 'max_depth': 9, 'min_samples_split': 20, 'min_samples_leaf': 6, 'max_features': 'sqrt'}. Best is trial 2 with value: 56.10061975866422.
[I 2024-05-07 21:38:52,457] Trial 3 finished with value: 56.804008168816225 and parameters: {'n_estimators': 600, 'max_depth': 12, 'min_

RandomForestRegressor RMSE: 55.92198476335225
Best Hyperparameters: {'n_estimators': 700, 'max_depth': 8, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 'sqrt'}


In [12]:
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import optuna
import numpy as np

def SVMRegressor_Optuna(X, y):
    def objective(trial, X, y):
        params = {
            'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid']),
            'C': trial.suggest_float('C', 1e-3, 1e3, log=True),
            'epsilon': trial.suggest_float('epsilon', 1e-3, 1e1, log=True),
        }
        if params['kernel'] == 'poly':
            params['degree'] = trial.suggest_int('degree', 2, 5)
        if params['kernel'] in ['poly', 'rbf', 'sigmoid']:
            params['gamma'] = trial.suggest_categorical('gamma', ['scale', 'auto'])

        svr = SVR(**params)
        mse_scores = -cross_val_score(svr, X, y, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
        rmse_scores = np.sqrt(mse_scores)
        return rmse_scores.mean()

    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective(trial, X, y), n_trials=100)
    trial = study.best_trial
    return trial

result = SVMRegressor_Optuna(X, y)
print(f"SVM Regressor RMSE: {result.value}")
print(f"Best Hyperparameters: {result.params}")


[I 2024-05-07 21:47:55,770] A new study created in memory with name: no-name-c37b2f32-c3f0-4bcd-8621-321d4597d1c7
[I 2024-05-07 21:47:55,903] Trial 0 finished with value: 58.167431049539346 and parameters: {'kernel': 'sigmoid', 'C': 1.6752444116415341, 'epsilon': 4.964190855400853, 'gamma': 'scale'}. Best is trial 0 with value: 58.167431049539346.
[I 2024-05-07 21:47:56,004] Trial 1 finished with value: 59.26255075968095 and parameters: {'kernel': 'linear', 'C': 51.98215643922212, 'epsilon': 0.022831187909983168}. Best is trial 0 with value: 58.167431049539346.
[I 2024-05-07 21:47:56,120] Trial 2 finished with value: 77.84862959111277 and parameters: {'kernel': 'linear', 'C': 0.14494296803363227, 'epsilon': 0.012827828941554275}. Best is trial 0 with value: 58.167431049539346.
[I 2024-05-07 21:47:56,251] Trial 3 finished with value: 77.56846144855092 and parameters: {'kernel': 'linear', 'C': 0.5241442657934577, 'epsilon': 2.7887376389862193}. Best is trial 0 with value: 58.167431049539

SVM Regressor RMSE: 54.92550687062436
Best Hyperparameters: {'kernel': 'sigmoid', 'C': 5.287514816452271, 'epsilon': 0.04013046105115052, 'gamma': 'scale'}


In [13]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import optuna
import numpy as np

def KNNRegressor_Optuna(X, y):
    def objective(trial, X, y):
        params = {
            'n_neighbors': trial.suggest_int('n_neighbors', 1, 20),
            'weights': trial.suggest_categorical('weights', ['uniform', 'distance']),
            'algorithm': trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute']),
            'p': trial.suggest_categorical('p', [1, 2])  # 1 for Manhattan distance, 2 for Euclidean distance
        }

        knn = KNeighborsRegressor(**params)
        mse_scores = -cross_val_score(knn, X, y, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
        rmse_scores = np.sqrt(mse_scores)
        return rmse_scores.mean()

    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective(trial, X, y), n_trials=100)
    trial = study.best_trial
    return trial

result = KNNRegressor_Optuna(X, y)
print(f"K-Nearest Neighbors Regressor RMSE: {result.value}")
print(f"Best Hyperparameters: {result.params}")

[I 2024-05-07 21:48:07,927] A new study created in memory with name: no-name-9071b9d1-3239-4b1e-b7ed-fb8ea28d5025
[I 2024-05-07 21:48:08,052] Trial 0 finished with value: 56.14288391252554 and parameters: {'n_neighbors': 20, 'weights': 'distance', 'algorithm': 'brute', 'p': 2}. Best is trial 0 with value: 56.14288391252554.
[I 2024-05-07 21:48:08,132] Trial 1 finished with value: 58.248116330620576 and parameters: {'n_neighbors': 17, 'weights': 'uniform', 'algorithm': 'ball_tree', 'p': 1}. Best is trial 0 with value: 56.14288391252554.
[I 2024-05-07 21:48:08,198] Trial 2 finished with value: 57.49779469027533 and parameters: {'n_neighbors': 10, 'weights': 'uniform', 'algorithm': 'ball_tree', 'p': 2}. Best is trial 0 with value: 56.14288391252554.
[I 2024-05-07 21:48:08,263] Trial 3 finished with value: 60.126361947859564 and parameters: {'n_neighbors': 5, 'weights': 'uniform', 'algorithm': 'auto', 'p': 2}. Best is trial 0 with value: 56.14288391252554.
[I 2024-05-07 21:48:08,341] Trial

K-Nearest Neighbors Regressor RMSE: 56.1108050250907
Best Hyperparameters: {'n_neighbors': 19, 'weights': 'distance', 'algorithm': 'kd_tree', 'p': 2}


# Mutliple Optuna Value

In [14]:
import optuna
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

def Optuna_Hyperparameter_tuning(X, y):
    def objective(trial, X, y):
        regressor = trial.suggest_categorical("regressor", ['RandomForest', 'XGB'])

        if regressor == "RandomForest":
            rf_params = {
                'n_estimators': trial.suggest_int("rf_n_estimators", 100, 1000, step=100),
                'max_depth': trial.suggest_int("rf_max_depth", 3, 20),
                'min_samples_split': trial.suggest_int("rf_min_samples_split", 2, 20),
                'min_samples_leaf': trial.suggest_int("rf_min_samples_leaf", 1, 20),
                'max_features': trial.suggest_categorical("rf_max_features", ['auto', 'sqrt', 'log2']),
                'random_state': 42,
                'n_jobs': -1
            }
            reg = RandomForestRegressor(**rf_params)

        else:
            xgb_params = {
                'n_estimators': trial.suggest_int('xgb_n_estimators', 100, 1000),
                'max_depth': trial.suggest_int('xgb_max_depth', 3, 10),
                'learning_rate': trial.suggest_float('xgb_learning_rate', 0.01, 0.3, log=True),
                'subsample': trial.suggest_float('xgb_subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('xgb_colsample_bytree', 0.5, 1.0),
                'gamma': trial.suggest_float('xgb_gamma', 1e-8, 1.0, log=True),
                'min_child_weight': trial.suggest_int('xgb_min_child_weight', 1, 300),
                'reg_alpha': trial.suggest_float('xgb_reg_alpha', 1e-8, 1.0, log=True),
                'reg_lambda': trial.suggest_float('xgb_reg_lambda', 1e-8, 1.0, log=True),
                'random_state': 42,
                'n_jobs': -1
            }
            reg = XGBRegressor(**xgb_params)

        mse_scores = -cross_val_score(reg, X, y, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
        rmse_scores = np.sqrt(mse_scores)
        return rmse_scores.mean()

    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective(trial=trial, X=X, y=y), n_trials=100)
    trial = study.best_trial
    return trial

result = Optuna_Hyperparameter_tuning(X=X, y=y)
print(f"RMSE: {result.value}")
print(f"Best Hyperparameters: {result.params}")

[I 2024-05-07 21:48:15,337] A new study created in memory with name: no-name-504c2edd-da1a-410e-94b6-c1d0c2c623f0
[I 2024-05-07 21:48:16,998] Trial 0 finished with value: 62.71735990816901 and parameters: {'regressor': 'XGB', 'xgb_n_estimators': 925, 'xgb_max_depth': 9, 'xgb_learning_rate': 0.05836496577801878, 'xgb_subsample': 0.7632792320844277, 'xgb_colsample_bytree': 0.8688973209926991, 'xgb_gamma': 3.2814662325504587e-07, 'xgb_min_child_weight': 22, 'xgb_reg_alpha': 0.0043633979933861726, 'xgb_reg_lambda': 0.004091505929515447}. Best is trial 0 with value: 62.71735990816901.
[I 2024-05-07 21:48:21,119] Trial 1 finished with value: 56.255784869261106 and parameters: {'regressor': 'RandomForest', 'rf_n_estimators': 500, 'rf_max_depth': 12, 'rf_min_samples_split': 7, 'rf_min_samples_leaf': 11, 'rf_max_features': 'sqrt'}. Best is trial 1 with value: 56.255784869261106.
[I 2024-05-07 21:48:25,976] Trial 2 finished with value: 56.00302852460477 and parameters: {'regressor': 'RandomFores

RMSE: 55.882372000971316
Best Hyperparameters: {'regressor': 'RandomForest', 'rf_n_estimators': 300, 'rf_max_depth': 18, 'rf_min_samples_split': 13, 'rf_min_samples_leaf': 5, 'rf_max_features': 'sqrt'}
